# Agent-Based Simulation 

In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
import random 
from matplotlib.collections import LineCollection
import matplotlib as mpl
import numpy as np
from scipy import optimize as opt
from scipy import integrate as intg
from scipy.optimize import least_squares
from scipy.stats import beta
from scipy.stats import cumfreq, beta
from utils import Agent, kill
%matplotlib inline
plt.style.use('../notebook.mplstyle')


## Exogenous Parameters


In [2]:
# Setting exogenous parameters
def reset_exog_params():
    global Bm, Bw, bm_vals, bw_vals, δ, um, uw, Fm, Fw, λm, λw, T  
    Bm = 10
    Bw = 10
    bm_vals = range(1,Bm+1) 
    bw_vals = range(1,Bw+1)
    
    δ = 0.97
    um = lambda θ : θ 
    uw = lambda θ : θ
    Fm = beta(2,4)
    Fw = beta(2,4)
    λm = 300
    λw = 30

    T = 40

## Steady State with Cutoff Strategies

In [3]:
def steady_state(μ, ω, verbose=False): 
    # Computing z's
    zm = []
    for b in bm_vals:
        if b==Bm:
            zm.append(1-δ*Fw.cdf(μ[Bm-1]))
        else:    
            z=1
            for i in range(1, Bm-b+1): 
                z *= ((δ*(1-Fw.cdf(μ[Bm-i])))/(1-δ*Fw.cdf(μ[Bm-i-1])))   
            zm.append(z)  
        
    zw = []
    for b in bw_vals:
        if b==Bw:
            zw.append(1-δ*Fw.cdf(ω[Bw-1]))
        else:    
            z=1
            for i in range(1, Bw-b+1): 
                z *= ((δ * (1-Fm.cdf(ω[Bw-i])))/(1-δ*Fm.cdf(ω[Bw-i-1])))
            zw.append(z)    
    
    # Computing steady-state masses
    Nm = λm * ((zm[Bm-1] - δ * zm[0] * (1 - Fw.cdf(μ[0]))) / ((1-δ) * zm[Bm-1]))
    Nw = λw * ((zw[Bw-1] - δ * zw[0] * (1 - Fm.cdf(ω[0]))) / ((1-δ) * zw[Bw-1]))   
    
    # Computing steady state distribution over budgets 
    Pbm = [(λm / (Nm * zm[Bm-1])) * zm[b] for b in range(Bm-1)]
    Pbw = [(λw / (Nw * zw[Bw-1])) * zw[b] for b in range(Bw-1)]
    Pbm.append((λm / (Nm * zm[Bm-1])))
    Pbw.append((λw / (Nw * zw[Bw-1]))) 
    
    # Computing tightness and alpha
    if Nw>Nm:
        τm = 1
    else:
        τm = Nw/Nm 
        
    τw = τm *(Nm/Nw) 
    αm = (τm*δ)/(1-δ*(1-τm))
    αw = (τw*δ)/(1-δ*(1-τw))
    return Nm, Nw, Pbm, Pbw, τm, τw, αm, αw

## Two-Sided Search Equilibrium Conditions 


In [4]:
# Optimality conditions
def SME(x): 
    # Compute steady state 
    Nm, Nw, Pbm, Pbw, τm, τw, αm, αw = steady_state(x[:Bm], x[Bm:])
    
    Um = um
    Uw = uw
    # Computing ex-ante expected utilities 
    
    # Initialysing system of equilibrium equations
    E = np.empty(2*Bm + 2*Bw + 2)
    
    # Initial conditions 
    E[0] = (Um(x[0]) 
            - αm * Um(x[0]) * Fw.cdf(x[0]) 
            - αm * intg.quad(lambda t: Um(t) * Fw.pdf(t), x[0], 1)[0]) 
    
    E[Bm] = (Uw(x[Bm]) 
            - αw * Uw(x[Bm]) * Fm.cdf(x[Bm]) 
            - αw * intg.quad(lambda t: Uw(t) * Fm.pdf(t), x[Bm], 1)[0]) 
    
    # Intertemporal optimality conditions for men
    for b in range(1,Bm):
        E[b] = (Um(x[b]) 
                - αm * Um(x[b]) * Fw.cdf(x[b]) 
                - αm * Um(x[b-1])*(1-Fw.cdf(x[b-1])) 
                - αm * intg.quad(lambda t : Um(t) * Fw.pdf(t), x[b], x[b-1])[0])
    
    # Intertemporal optimality conditions for women
    for b in range(1,Bw):
        E[Bm+b] = (Uw(x[Bm+b]) 
                - αw * Uw(x[Bm+b]) * Fm.cdf(x[Bm+b]) 
                - αw * Uw(x[Bm+b-1])*(1-Fm.cdf(x[Bm+b-1])) 
                - αw * intg.quad(lambda t : Uw(t) * Fm.pdf(t), x[Bm+b], x[Bm+b-1])[0])
        
    # PMF unity sum conditions
    E[Bm+Bw] = sum(Pbm)-1
    E[Bm+Bw+1] = sum(Pbw)-1
    
    # PMF non-negativity conditions
    for b in range(Bm):
        E[Bm+Bw+2+b] = Pbm[b]-abs(Pbm[b])
        
    for b in range(Bw): 
        E[Bm+Bw+2+Bm+b] = Pbw[b]-abs(Pbw[b])    
        
    return E 

## Solving For Steady State Equilibria 

In [5]:
reset_exog_params()
m_test = np.random.rand(Bm)#*0.5
w_test = np.random.rand(Bw)#*0.5  

print('μ0: ', m_test)
print('ω0: ', w_test)
print('')

x_start = np.concatenate((m_test, w_test), axis=None)
solution = least_squares(SME, x_start, bounds = (0,1), verbose=1) 

μ_star = solution.x[:Bm]
ω_star = solution.x[Bm:]
print('')
print('μ*', μ_star) 
print('ω*', ω_star) 
print('Loss:', round(solution.cost,6))

μ0:  [0.57983099 0.34372093 0.03989875 0.48676351 0.49284925 0.47067396
 0.51839983 0.09991654 0.21514808 0.00722934]
ω0:  [0.21509501 0.9160303  0.58992957 0.21105256 0.18502915 0.60641744
 0.6389585  0.25686722 0.31585936 0.88081957]

`gtol` termination condition is satisfied.
Function evaluations 11, initial cost 1.6633e-01, final cost 9.0466e-24, first-order optimality 4.03e-13.

μ* [0.35380838 0.26147746 0.2056276  0.1662619  0.1365317  0.11319741
 0.09444242 0.07913146 0.06649918 0.05600003]
ω* [0.526524   0.45591668 0.41087663 0.37729289 0.35035712 0.32781029
 0.30839844 0.29134882 0.27614945 0.26244259]
Loss: 0.0


## Agent-Based Simulation

In [6]:
men = []
women = [] 

for t in range(0,T):
    # Add new agents 
    for i, θ in enumerate(Fm.rvs(size=λm)):
        id = t*λm+i
        men.append(Agent(id, t, θ, Bm, μ_star))

    for i, θ in enumerate(Fw.rvs(size=λw)):
        id = t*λw+i
        women.append(Agent(id, t, θ, Bm, ω_star))     

    # Pairings 
    random.shuffle(men)
    random.shuffle(women) 
    for pair in zip(men, women):
        m, w = pair
        am = m.swipe(w)
        aw = w.swipe(m)
        m.update(am, aw, w, t)
        w.update(aw, am, m, t)

        if m.b == 0:
            kill(m, men, t)
        if w.b ==0:
            kill(w, women, t)     
    # Departures 
    for m in random.sample(men, round((1-δ)*len(men))):
        print(f'Departing: man {m.id}')
        kill(m, men, t)
    for w in random.sample(women, round((1-δ)*len(women))):
        print(f'Departing: woman {w.id}')
        kill(w, women, t)    
    





killing men 70
killing men 194
killing men 226
killing men 223
killing men 248
killing men 277
killing men 278
killing men 53
killing men 211
killing women 28
killing men 511
killing men 586
killing men 286
killing men 327
killing men 303
killing men 346
killing men 292
killing men 279
killing men 150
killing men 397
killing men 35
killing men 165
killing men 427
killing men 450
killing men 146
killing men 533
killing men 128
killing men 342
killing women 58
killing women 56
killing men 465
killing men 803
killing men 483
killing men 89
killing men 444
killing men 175
killing men 456
killing men 315
killing men 784
killing men 274
killing men 519
killing men 590
killing men 895
killing men 510
killing men 564
killing men 524
killing men 145
killing men 179
killing men 489
killing men 517
killing men 392
killing men 583
killing men 887
killing men 629
killing men 280
killing men 364
killing women 34
killing women 40
killing women 61
killing men 910
killing men 952
killing men 715
killin